In [2]:
import requests
from bs4 import BeautifulSoup
import pandas

def get_movie_info(start, end):
    
    results = []
    
    for i in range(start, end+1):
        url=f'https://movie.naver.com/movie/point/af/list.nhn?page={i}'
        r = requests.get(url)
        bs = BeautifulSoup(r.text, 'lxml')

        trs = bs.select('table.list_netizen > tbody > tr')
        for tr in trs:
            tds = tr.select('td')
            #if len(tds) != 5:
            #    continue
            number = tds[0].text
            point = tds[1].select('em')[0].text
            movie = tds[1].select('a')[0].text
            #writer = tds[2].select('a')[0].text
            date = tds[2].text

#             results.append({
#                 'number': number,
#                 'movie': movie,
#                 'point': point,
#                 'date': date
#             })

            # pandas는 리스트를 처리
            results.append([movie, point, date])
    return results

column = ['영화제목', '평점', '작성자 & 날짜']
results = get_movie_info(1, 3)

dataframe = pandas.DataFrame(results, columns=column)
#print(dataframe)

# Excel로 저장
dataframe.to_excel('movie.xlsx', sheet_name='Naver Movie', header=True, startrow=1)


## 개선 코드 (엑셀 저장 제외)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_movie_point(start, end=1):
    results = []
    for i in range(start, end+1):
        url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'.format(i)
        r = requests.get(url)
        bs = BeautifulSoup(r.text, "lxml")
        trs = bs.select("table.list_netizen > tbody > tr")
        for tr in trs:  #다수의 평점
            # 번호
            number = tr.select_one("td.ac.num").text
            # 작성자
            writer = tr.select_one("td.num > a.author").text
            
            # td 의 title 클래스를 구합니다.
            # title, point 둘다 td.title 하위에 있기 때문에 연산비용을 줄이기 위해 td.title로 한번에
            tr_data = tr.select_one("td.title")

            # td class="title" 자식중 최초 a 태그안에 제목만 추출
            title = tr_data.select_one("a").text

            # td class="title" 자식중 div 태그 자식중 em 태그에 점수 추출
            point = tr_data.select_one("div.list_netizen_score > em").text

            # td class="title" 태그에서 a, div, br 태그 제거
            # extract() 함수는 태그와 태그의 내용까지 모두 제거합니다.
            [x.extract() for x in tr_data.select("a")]
            [x.extract() for x in tr_data.select("div")]
            [x.extract() for x in tr_data.select("br")]

            # 위에서 태그를 모두 제거한 tr_data에서 내용만 추출
            content = tr_data.text.strip()
            results.append({
                "number": number,
                "movie": title,
                "point": point,
                "writer": writer,
                "contents": content,
            })
    return results

print(get_movie_point(1,1))